# DeepRoof Inference — Roof Layout + Geometry + 3D

**Per roof plane:** instance mask, polygon, class (flat/sloped), slope angle, azimuth, normal vector.

**Visualizations:** semantic map, instance color map, normal map (RGB), overlay with slope labels, **interactive 3D roof model**.

In [ ]:
from __future__ import annotations

import json, os, sys, glob
from pathlib import Path

import cv2
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import numpy as np
import torch
import torch.nn.functional as F

In [ ]:
# ======================== CONFIGURATION ========================
MODEL_INPUT_SIZE = 512
MASK_THRESHOLD = 0.4    # Mask sigmoid threshold for roof vs background
SCORE_THRESHOLD = 0.1   # Min combined score to keep a query as an instance
MIN_INSTANCE_AREA = 50  # Min pixels for a valid instance (at 128x128 mask res)


def detect_project_root() -> Path:
    for c in [Path.cwd(), Path.cwd().parent, Path('/workspace/roof'), Path('/Users/voskan/Desktop/DeepRoof-2026')]:
        if (c / 'configs').exists() and (c / 'deeproof').exists():
            return c
    raise FileNotFoundError('Could not auto-detect project root.')


PROJECT_ROOT = detect_project_root()
CONFIG_PATH = PROJECT_ROOT / 'configs' / 'deeproof_production_swin_L.py'

# Check fine-tune dir first, then scratch
WORK_DIR = PROJECT_ROOT / 'work_dirs' / 'swin_l_finetune_v2'
if not WORK_DIR.exists():
    WORK_DIR = PROJECT_ROOT / 'work_dirs' / 'swin_l_scratch_v1'

# Auto-pick latest checkpoint
last_ckpt_ptr = WORK_DIR / 'last_checkpoint'
if last_ckpt_ptr.exists():
    t = Path(last_ckpt_ptr.read_text().strip())
    CHECKPOINT_PATH = t if t.is_absolute() else WORK_DIR / t
else:
    ckpts = sorted(WORK_DIR.glob('iter_*.pth'))
    CHECKPOINT_PATH = ckpts[-1] if ckpts else WORK_DIR / 'iter_16000.pth'

# ---- INPUT IMAGE ----
# Option A: external image
INPUT_IMAGE_PATH = Path('/workspace/test.png')

# Option B: auto-find first OmniCity val image (uncomment next 3 lines)
# val_txt = PROJECT_ROOT / 'data' / 'OmniCity' / 'val.txt'
# first_val = val_txt.read_text().strip().split('\n')[0].strip()
# INPUT_IMAGE_PATH = PROJECT_ROOT / 'data' / 'OmniCity' / 'images' / f'{first_val}.jpg'

if not INPUT_IMAGE_PATH.exists():
    fallback = PROJECT_ROOT / 'test.png'
    if fallback.exists():
        INPUT_IMAGE_PATH = fallback

OUTPUT_DIR = PROJECT_ROOT / 'outputs' / 'checkpoint_inference'
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

DEVICE = 'cuda:0' if torch.cuda.is_available() else 'cpu'
print(f'CHECKPOINT: {CHECKPOINT_PATH}')
print(f'INPUT: {INPUT_IMAGE_PATH}')
print(f'MASK_THRESHOLD: {MASK_THRESHOLD}, SCORE_THRESHOLD: {SCORE_THRESHOLD}')

In [ ]:
# ======================== IMAGE PREPROCESSING ========================
for p in (CONFIG_PATH, CHECKPOINT_PATH, INPUT_IMAGE_PATH):
    if not p.exists():
        raise FileNotFoundError(f'Not found: {p}')

img_orig_bgr = cv2.imread(str(INPUT_IMAGE_PATH), cv2.IMREAD_COLOR)
orig_h, orig_w = img_orig_bgr.shape[:2]

# Center-crop to square, resize to training resolution
crop_size = min(orig_h, orig_w)
y0 = (orig_h - crop_size) // 2
x0 = (orig_w - crop_size) // 2
img_bgr = cv2.resize(img_orig_bgr[y0:y0+crop_size, x0:x0+crop_size],
                      (MODEL_INPUT_SIZE, MODEL_INPUT_SIZE), interpolation=cv2.INTER_AREA)
PREPROCESSED_PATH = OUTPUT_DIR / 'preprocessed.png'
cv2.imwrite(str(PREPROCESSED_PATH), img_bgr)

img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
H, W = MODEL_INPUT_SIZE, MODEL_INPUT_SIZE
print(f'{orig_w}x{orig_h} -> crop {crop_size} -> {H}x{W}')

In [ ]:
# ======================== LOAD MODEL ========================
os.environ.setdefault('TORCH_FORCE_NO_WEIGHTS_ONLY_LOAD', '1')
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

from mmseg.utils import register_all_modules
from mmseg.apis import init_model
from mmengine.config import ConfigDict
from mmengine.dataset import Compose

register_all_modules(init_default_scope=False)
import deeproof.models.backbones.swin_v2_compat
import deeproof.models.deeproof_model
import deeproof.models.heads.mask2former_head
import deeproof.models.heads.geometry_head
import deeproof.models.losses

model = init_model(str(CONFIG_PATH), str(CHECKPOINT_PATH), device=DEVICE)
model.test_cfg = ConfigDict(dict(mode='whole'))
model.eval()
print(f'Model loaded. geometry_head: {hasattr(model, "geometry_head")}')

In [ ]:
# ======================== FORWARD PASS ========================
test_pipeline = model.cfg.get('test_pipeline', [
    dict(type='LoadImageFromFile'),
    dict(type='Resize', scale=(MODEL_INPUT_SIZE, MODEL_INPUT_SIZE), keep_ratio=False),
    dict(type='PackSegInputs'),
])
pipeline = Compose(test_pipeline)
data = pipeline(dict(img_path=str(PREPROCESSED_PATH)))
data_batch = model.data_preprocessor(
    dict(inputs=[data['inputs']], data_samples=[data['data_samples']]), False
)

with torch.no_grad():
    x = model.extract_feat(data_batch['inputs'])
    all_cls_scores, all_mask_preds = model.decode_head(x, data_batch['data_samples'])

    # GeometryHead: surface normals per query
    query_emb = getattr(model.decode_head, 'last_query_embeddings', None)
    geo_preds = None
    if query_emb is not None and hasattr(model, 'geometry_head'):
        qe = query_emb
        if isinstance(qe, (list, tuple)): qe = qe[-1]
        if qe.ndim == 4: qe = qe[-1]
        if qe.ndim == 2: qe = qe.unsqueeze(0)
        geo_preds = model.geometry_head(qe)  # [1, Q, 3]

# Last decoder layer
cls_scores = all_cls_scores[-1][0]   # [Q, C+1]
mask_logits = all_mask_preds[-1][0]  # [Q, h, w]
mask_h, mask_w = mask_logits.shape[-2:]

cls_probs = torch.softmax(cls_scores, dim=-1)   # [Q, C+1]
obj_probs = cls_probs[:, :-1]                     # [Q, C] (exclude no-obj)
no_obj_probs = cls_probs[:, -1]                    # [Q]
mask_sigmoid = mask_logits.sigmoid()               # [Q, h, w]

Q = cls_scores.shape[0]
C = obj_probs.shape[1]
print(f'Queries: {Q}, Classes: {C}, Mask res: {mask_h}x{mask_w}')
if geo_preds is not None:
    print(f'GeometryHead: {geo_preds.shape}')

In [ ]:
# ======================== PER-QUERY INSTANCE EXTRACTION ========================
# Each Mask2Former query IS a roof plane instance.
# We use the query's mask, class, and geometry directly — no connected components.

query_normals = geo_preds[0].cpu().numpy() if geo_preds is not None else None  # [Q, 3]

instances = []
for qi in range(Q):
    # Query class and confidence
    best_cls = int(obj_probs[qi].argmax())
    best_cls_prob = float(obj_probs[qi, best_cls])
    p_no_obj = float(no_obj_probs[qi])

    # Binary mask at low resolution
    mask_q = mask_sigmoid[qi]  # [h, w]
    binary_mask_lr = (mask_q > MASK_THRESHOLD)  # [h, w] bool
    area_lr = int(binary_mask_lr.sum())

    # Filter: skip empty masks and low-confidence queries
    if area_lr < MIN_INSTANCE_AREA:
        continue

    # Combined score: class confidence × (1 - no_obj) × mean mask in active region
    mean_mask_in_region = float(mask_q[binary_mask_lr].mean())
    combined_score = best_cls_prob * (1.0 - p_no_obj) * mean_mask_in_region
    if combined_score < SCORE_THRESHOLD:
        continue

    # Upscale mask to full resolution
    mask_full = F.interpolate(
        binary_mask_lr.float().unsqueeze(0).unsqueeze(0),
        size=(H, W), mode='nearest'
    )[0, 0].bool().cpu().numpy()
    area_full = int(mask_full.sum())

    # Geometry from GeometryHead
    normal = None
    slope_deg = None
    azimuth_deg = None
    if query_normals is not None:
        normal = query_normals[qi]  # [3] = (nx, ny, nz)
        nz = float(np.clip(abs(normal[2]), -1.0, 1.0))
        slope_deg = float(np.degrees(np.arccos(nz)))
        azimuth_deg = float(np.degrees(np.arctan2(normal[1], normal[0]))) % 360

    class_name = {0: 'background', 1: 'flat_roof', 2: 'sloped_roof'}.get(best_cls, f'cls_{best_cls}')

    instances.append({
        'query_id': qi,
        'mask': mask_full,
        'class_id': best_cls,
        'class_name': class_name,
        'score': round(combined_score, 4),
        'cls_prob': round(best_cls_prob, 4),
        'no_obj_prob': round(p_no_obj, 4),
        'area_px': area_full,
        'normal': normal.tolist() if normal is not None else None,
        'slope_deg': round(slope_deg, 1) if slope_deg is not None else None,
        'azimuth_deg': round(azimuth_deg, 0) if azimuth_deg is not None else None,
    })

# Sort by area (largest first)
instances.sort(key=lambda x: x['area_px'], reverse=True)

print(f'\n=== {len(instances)} ROOF PLANE INSTANCES ===')
print(f'{"#":>3} {"Query":>5} {"Class":>12} {"Slope":>6} {"Az":>5} {"Normal":>22} {"Score":>6} {"Area":>7}')
print('-' * 75)
for i, inst in enumerate(instances):
    n_str = f'({inst["normal"][0]:+.2f},{inst["normal"][1]:+.2f},{inst["normal"][2]:+.2f})' if inst['normal'] else 'N/A'
    s_str = f'{inst["slope_deg"]:.0f}\u00b0' if inst['slope_deg'] is not None else 'N/A'
    a_str = f'{inst["azimuth_deg"]:.0f}\u00b0' if inst['azimuth_deg'] is not None else 'N/A'
    print(f'{i:3d} Q{inst["query_id"]:>4d} {inst["class_name"]:>12} {s_str:>6} {a_str:>5} {n_str:>22} {inst["score"]:6.3f} {inst["area_px"]:7d}')

In [ ]:
# ======================== BUILD MAPS ========================

# 1. Semantic map (per-pixel class)
sem_map = np.zeros((H, W), dtype=np.uint8)

# 2. Instance map (per-pixel instance ID, 0 = background)
inst_map = np.zeros((H, W), dtype=np.int32)

# 3. Normal map (per-pixel RGB normal)
normal_map = np.zeros((H, W, 3), dtype=np.float32)  # nx, ny, nz
normal_map[:, :, 2] = 1.0  # default: pointing up

# 4. Slope map (per-pixel slope in degrees)
slope_map = np.zeros((H, W), dtype=np.float32)

# Paint instances (largest first, so smaller instances overlay larger ones)
for i, inst in enumerate(instances):
    mask = inst['mask']
    sem_map[mask] = inst['class_id']
    inst_map[mask] = i + 1  # 1-indexed
    if inst['normal'] is not None:
        normal_map[mask] = inst['normal']
    if inst['slope_deg'] is not None:
        slope_map[mask] = inst['slope_deg']

# Stats
unique_cls = np.unique(sem_map).tolist()
class_areas = {int(c): float((sem_map == c).sum()) / (H * W) for c in unique_cls}
print(f'Semantic classes: {unique_cls}, areas: {class_areas}')
print(f'Roof coverage: {float((sem_map > 0).sum()) / (H * W):.1%}')
print(f'Unique instances: {len(np.unique(inst_map)) - 1}')

In [ ]:
# ======================== VISUALIZATION (4 panels) ========================

# --- Panel 1: Input image ---
# (already have img_rgb)

# --- Panel 2: Semantic map ---
sem_palette = np.array([[0,0,0], [0,200,0], [220,50,50]], dtype=np.uint8)
sem_vis = sem_palette[np.clip(sem_map, 0, 2)]

# --- Panel 3: Instance color map (each facet unique color) ---
np.random.seed(42)
n_inst = len(instances)
inst_colors = np.random.randint(60, 255, size=(n_inst + 1, 3), dtype=np.uint8)
inst_colors[0] = [0, 0, 0]  # background = black
inst_vis = inst_colors[np.clip(inst_map, 0, n_inst)]

# --- Panel 4: Normal map (nx,ny,nz -> RGB) ---
# Map [-1,1] -> [0,255]:  R = nx, G = ny, B = nz
normal_vis = ((normal_map + 1.0) * 0.5 * 255).clip(0, 255).astype(np.uint8)
# Only show on roof pixels
normal_vis[sem_map == 0] = 0

fig, axes = plt.subplots(2, 2, figsize=(16, 16))

axes[0, 0].imshow(img_rgb)
axes[0, 0].set_title('Input image')
axes[0, 0].axis('off')

axes[0, 1].imshow(sem_vis)
patches = [
    mpatches.Patch(color=[0,0,0], label='Background'),
    mpatches.Patch(color=np.array([0,200,0])/255, label=f'Flat roof ({class_areas.get(1,0):.0%})'),
    mpatches.Patch(color=np.array([220,50,50])/255, label=f'Sloped roof ({class_areas.get(2,0):.0%})'),
]
axes[0, 1].legend(handles=patches, loc='lower right', fontsize=9)
axes[0, 1].set_title('Semantic map')
axes[0, 1].axis('off')

axes[1, 0].imshow(inst_vis)
axes[1, 0].set_title(f'Instance map ({len(instances)} planes)')
axes[1, 0].axis('off')

axes[1, 1].imshow(normal_vis)
axes[1, 1].set_title('Normal map (R=nx, G=ny, B=nz)')
axes[1, 1].axis('off')

plt.tight_layout()
plt.savefig(str(OUTPUT_DIR / '4panel_result.png'), dpi=150, bbox_inches='tight')
plt.show()
print(f'Saved: {OUTPUT_DIR / "4panel_result.png"}')

In [ ]:
# ======================== OVERLAY WITH SLOPE LABELS ========================
overlay = cv2.addWeighted(img_rgb, 0.55, sem_vis, 0.45, 0.0)

# Draw instance boundaries and slope labels
roof_polygons = []
for i, inst in enumerate(instances):
    contours, _ = cv2.findContours(inst['mask'].astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    for contour in contours:
        area = float(cv2.contourArea(contour))
        if area < 50:
            continue
        epsilon = 0.003 * cv2.arcLength(contour, True)
        poly = cv2.approxPolyDP(contour, epsilon, True).reshape(-1, 2)
        if poly.shape[0] < 3:
            continue

        # Draw boundary
        pts = poly.reshape(-1, 1, 2)
        cv2.polylines(overlay, [pts], True, (255, 255, 255), 1, cv2.LINE_AA)

        # Centroid for label
        m = cv2.moments(contour)
        if m['m00'] > 0:
            cx, cy = int(m['m10']/m['m00']), int(m['m01']/m['m00'])
        else:
            cx, cy = int(poly[0, 0]), int(poly[0, 1])

        # Label: slope angle
        if inst['slope_deg'] is not None and area > 200:
            txt = f"{inst['slope_deg']:.0f}\u00b0"
            cv2.putText(overlay, txt, (cx-10, cy+4), cv2.FONT_HERSHEY_SIMPLEX, 0.35, (255, 255, 0), 1, cv2.LINE_AA)

        roof_polygons.append({
            'polygon_id': len(roof_polygons),
            'instance_id': i,
            'query_id': inst['query_id'],
            'class_id': inst['class_id'],
            'class_name': inst['class_name'],
            'score': inst['score'],
            'area_px': area,
            'slope_deg': inst['slope_deg'],
            'azimuth_deg': inst['azimuth_deg'],
            'normal': inst['normal'],
            'points_xy': poly.astype(int).tolist(),
        })

cv2.imwrite(str(OUTPUT_DIR / 'overlay.png'), cv2.cvtColor(overlay, cv2.COLOR_RGB2BGR))
cv2.imwrite(str(OUTPUT_DIR / 'semantic_mask.png'), sem_map)

plt.figure(figsize=(12, 12))
plt.imshow(overlay)
plt.title(f'{len(roof_polygons)} roof polygons with slope angles')
plt.axis('off')
plt.show()
print(f'Total polygons: {len(roof_polygons)}')

## 🏗️ 3D Roof Visualization

Each detected roof facet is rendered as a **3D tilted plane** based on its predicted normal vector `(nx, ny, nz)`.

The Z-height is computed from the plane equation:
$$z(x, y) = -\frac{n_x \cdot (x - c_x) + n_y \cdot (y - c_y)}{n_z} + z_{base}$$

In [ ]:
# ======================== 3D ROOF VISUALIZATION ========================
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from matplotlib.colors import to_rgba

# Scale factor: how much to exaggerate the Z (height) for visibility.
# Roofs are relatively shallow, so we amplify Z to make slopes visible.
Z_EXAGGERATION = 2.0

# Base height for stacked roof planes (taller buildings get higher base)
BASE_HEIGHT = 5.0

fig = plt.figure(figsize=(16, 12))
ax = fig.add_subplot(111, projection='3d')

# Use the same instance colors as the 2D visualization
np.random.seed(42)
colors_3d = np.random.rand(len(instances) + 1, 3)
colors_3d[0] = [0.2, 0.2, 0.2]  # background

# Overlay the input image on the ground plane (Z=0)
# Downsample for performance
ds = 4  # downsample factor
img_small = img_rgb[::ds, ::ds]
h_s, w_s = img_small.shape[:2]
x_ground = np.linspace(0, W, w_s)
y_ground = np.linspace(0, H, h_s)
X_g, Y_g = np.meshgrid(x_ground, y_ground)
Z_g = np.zeros_like(X_g)

# Plot ground image as a surface
ax.plot_surface(
    X_g, Y_g, Z_g,
    facecolors=img_small / 255.0,
    rstride=1, cstride=1,
    shade=False, alpha=0.4
)

# Render each roof facet as a 3D polygon
for i, inst in enumerate(instances):
    if inst['normal'] is None:
        continue

    nx_n, ny_n, nz_n = inst['normal']
    # Ensure nz isn't zero (avoid div-by-zero for vertical surfaces)
    if abs(nz_n) < 0.01:
        nz_n = 0.01 if nz_n >= 0 else -0.01

    # Get the contour polygon for this instance
    contours, _ = cv2.findContours(
        inst['mask'].astype(np.uint8),
        cv2.RETR_EXTERNAL,
        cv2.CHAIN_APPROX_SIMPLE
    )

    for contour in contours:
        area = cv2.contourArea(contour)
        if area < 100:
            continue

        # Simplify polygon for rendering
        epsilon = 0.01 * cv2.arcLength(contour, True)
        poly_2d = cv2.approxPolyDP(contour, epsilon, True).reshape(-1, 2)  # (N, 2)
        if poly_2d.shape[0] < 3:
            continue

        # Centroid of this polygon
        cx = poly_2d[:, 0].mean()
        cy = poly_2d[:, 1].mean()

        # Compute Z for each vertex using the plane equation:
        # z = -(nx*(x-cx) + ny*(y-cy)) / nz + z_base
        xs = poly_2d[:, 0].astype(float)
        ys = poly_2d[:, 1].astype(float)
        zs = -(nx_n * (xs - cx) + ny_n * (ys - cy)) / nz_n
        zs = zs * Z_EXAGGERATION + BASE_HEIGHT

        # Build 3D polygon vertices: (x, y, z)
        verts = list(zip(xs, ys, zs))

        # Draw the 3D polygon
        color = colors_3d[i + 1]
        face = Poly3DCollection(
            [verts],
            alpha=0.7,
            facecolors=[to_rgba(color, alpha=0.7)],
            edgecolors=['white'],
            linewidths=0.5
        )
        ax.add_collection3d(face)

        # Also draw vertical walls from ground to roof edge
        for j in range(len(verts)):
            j_next = (j + 1) % len(verts)
            wall = [
                (xs[j], ys[j], 0),
                (xs[j_next], ys[j_next], 0),
                (xs[j_next], ys[j_next], zs[j_next]),
                (xs[j], ys[j], zs[j]),
            ]
            wall_face = Poly3DCollection(
                [wall],
                alpha=0.15,
                facecolors=[to_rgba(color, alpha=0.15)],
                edgecolors=[to_rgba(color, alpha=0.3)],
                linewidths=0.3
            )
            ax.add_collection3d(wall_face)

# Set axis properties
ax.set_xlim(0, W)
ax.set_ylim(0, H)
ax.set_zlim(0, BASE_HEIGHT * 3)
ax.set_xlabel('X (pixels)')
ax.set_ylabel('Y (pixels)')
ax.set_zlabel('Height (exaggerated)')

# Invert Y so image coordinates match
ax.invert_yaxis()

# Set viewing angle
ax.view_init(elev=45, azim=-60)

plt.title(f'3D Roof Model — {len(instances)} facets (Z exaggeration: {Z_EXAGGERATION}x)', fontsize=14)
plt.tight_layout()
plt.savefig(str(OUTPUT_DIR / '3d_roof_model.png'), dpi=150, bbox_inches='tight')
plt.show()
print(f'\n✅ 3D visualization saved: {OUTPUT_DIR / "3d_roof_model.png"}')

In [ ]:
# ======================== ADDITIONAL 3D VIEWS ========================
# Show the same model from multiple angles for better understanding

fig, axes_3d = plt.subplots(1, 3, figsize=(24, 8), subplot_kw={'projection': '3d'})

views = [
    (60, -45, 'Front-Right View'),
    (30, -135, 'Back-Left View'),
    (90, -90, 'Top-Down View'),
]

for ax_v, (elev, azim, title) in zip(axes_3d, views):
    # Redraw roof facets on each subplot
    for i, inst in enumerate(instances):
        if inst['normal'] is None:
            continue

        nx_n, ny_n, nz_n = inst['normal']
        if abs(nz_n) < 0.01:
            nz_n = 0.01 if nz_n >= 0 else -0.01

        contours, _ = cv2.findContours(
            inst['mask'].astype(np.uint8),
            cv2.RETR_EXTERNAL,
            cv2.CHAIN_APPROX_SIMPLE
        )

        for contour in contours:
            if cv2.contourArea(contour) < 100:
                continue
            epsilon = 0.01 * cv2.arcLength(contour, True)
            poly_2d = cv2.approxPolyDP(contour, epsilon, True).reshape(-1, 2)
            if poly_2d.shape[0] < 3:
                continue

            cx = poly_2d[:, 0].mean()
            cy = poly_2d[:, 1].mean()
            xs = poly_2d[:, 0].astype(float)
            ys = poly_2d[:, 1].astype(float)
            zs = -(nx_n * (xs - cx) + ny_n * (ys - cy)) / nz_n
            zs = zs * Z_EXAGGERATION + BASE_HEIGHT
            verts = list(zip(xs, ys, zs))

            color = colors_3d[i + 1]
            face = Poly3DCollection(
                [verts], alpha=0.7,
                facecolors=[to_rgba(color, alpha=0.7)],
                edgecolors=['white'], linewidths=0.5
            )
            ax_v.add_collection3d(face)

    ax_v.set_xlim(0, W)
    ax_v.set_ylim(0, H)
    ax_v.set_zlim(0, BASE_HEIGHT * 3)
    ax_v.invert_yaxis()
    ax_v.view_init(elev=elev, azim=azim)
    ax_v.set_title(title, fontsize=11)
    ax_v.set_xlabel('X'); ax_v.set_ylabel('Y'); ax_v.set_zlabel('Z')

plt.suptitle('3D Roof Model — Multi-Angle Views', fontsize=14)
plt.tight_layout()
plt.savefig(str(OUTPUT_DIR / '3d_roof_multiview.png'), dpi=150, bbox_inches='tight')
plt.show()
print(f'Saved: {OUTPUT_DIR / "3d_roof_multiview.png"}')

In [ ]:
# ======================== TOP QUERY MASKS ========================
# Show the 8 queries with largest mask area
query_areas = [(qi, float((mask_sigmoid[qi] > MASK_THRESHOLD).float().sum())) for qi in range(Q)]
query_areas.sort(key=lambda x: x[1], reverse=True)
top_queries = [qa[0] for qa in query_areas[:8] if qa[1] > 0]

if top_queries:
    n_show = len(top_queries)
    cols = 4
    rows = (n_show + cols - 1) // cols
    fig, axes = plt.subplots(rows, cols, figsize=(16, 4*rows))
    axes = np.array(axes).flatten()
    for idx, qi in enumerate(top_queries):
        mask_vis = mask_sigmoid[qi].cpu().numpy()
        cov = float((mask_sigmoid[qi] > MASK_THRESHOLD).float().sum())
        cls = int(obj_probs[qi].argmax())
        conf = float(obj_probs[qi].max())
        n_str = ''
        if query_normals is not None:
            nz = abs(float(query_normals[qi][2]))
            slope = float(np.degrees(np.arccos(np.clip(nz, -1, 1))))
            n_str = f', {slope:.0f}\u00b0'
        axes[idx].imshow(mask_vis, cmap='hot', vmin=0, vmax=1)
        axes[idx].set_title(f'Q{qi} cls={cls} P={conf:.2f}{n_str}\narea={cov:.0f}px', fontsize=9)
        axes[idx].axis('off')
    for idx in range(len(top_queries), len(axes)):
        axes[idx].axis('off')
    plt.suptitle('Top query masks (sigmoid, hot colormap)', fontsize=13)
    plt.tight_layout()
    plt.show()

In [ ]:
# ======================== SAVE JSON + GEOJSON ========================
json_polygons = [{k: v for k, v in p.items()} for p in roof_polygons]

with (OUTPUT_DIR / 'roof_polygons.json').open('w') as f:
    json.dump({
        'image': str(INPUT_IMAGE_PATH),
        'checkpoint': str(CHECKPOINT_PATH),
        'model_input_size': MODEL_INPUT_SIZE,
        'mask_threshold': MASK_THRESHOLD,
        'total_instances': len(instances),
        'total_polygons': len(roof_polygons),
        'polygons': json_polygons,
    }, f, indent=2)

# GeoJSON (pixel-space)
features = []
for p in roof_polygons:
    coords = [[float(x), float(y)] for x, y in p['points_xy']]
    if coords and coords[0] != coords[-1]:
        coords.append(coords[0])
    features.append({
        'type': 'Feature',
        'properties': {k: v for k, v in p.items() if k != 'points_xy'},
        'geometry': {'type': 'Polygon', 'coordinates': [coords]},
    })

with (OUTPUT_DIR / 'roof_polygons.geojson').open('w') as f:
    json.dump({'type': 'FeatureCollection', 'features': features}, f, indent=2)

# OBJ Export (3D mesh)
obj_path = OUTPUT_DIR / 'roof_model.obj'
with open(str(obj_path), 'w') as f:
    f.write('# DeepRoof 3D Roof Model\n')
    f.write(f'# {len(instances)} roof facets\n\n')
    vert_offset = 1  # OBJ is 1-indexed
    for i, inst in enumerate(instances):
        if inst['normal'] is None:
            continue
        nx_n, ny_n, nz_n = inst['normal']
        if abs(nz_n) < 0.01:
            nz_n = 0.01
        contours, _ = cv2.findContours(
            inst['mask'].astype(np.uint8),
            cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE
        )
        for contour in contours:
            if cv2.contourArea(contour) < 100:
                continue
            epsilon = 0.01 * cv2.arcLength(contour, True)
            poly_2d = cv2.approxPolyDP(contour, epsilon, True).reshape(-1, 2)
            if poly_2d.shape[0] < 3:
                continue
            cx = poly_2d[:, 0].mean()
            cy = poly_2d[:, 1].mean()
            xs = poly_2d[:, 0].astype(float)
            ys = poly_2d[:, 1].astype(float)
            zs = -(nx_n * (xs - cx) + ny_n * (ys - cy)) / nz_n + BASE_HEIGHT
            f.write(f'# Facet {i} ({inst["class_name"]}) slope={inst["slope_deg"]}deg\n')
            f.write(f'vn {nx_n:.4f} {ny_n:.4f} {nz_n:.4f}\n')
            for x, y, z in zip(xs, ys, zs):
                f.write(f'v {x:.2f} {y:.2f} {z:.4f}\n')
            n_verts = len(xs)
            face_indices = ' '.join(str(vert_offset + j) for j in range(n_verts))
            f.write(f'f {face_indices}\n\n')
            vert_offset += n_verts

# Summary
summary = {
    'checkpoint': str(CHECKPOINT_PATH),
    'input': str(INPUT_IMAGE_PATH),
    'instances': len(instances),
    'polygons': len(roof_polygons),
    'classes': unique_cls,
    'roof_coverage': round(float((sem_map > 0).sum()) / (H * W), 4),
}
print(json.dumps(summary, indent=2))
print(f'\nAll outputs saved to: {OUTPUT_DIR}')
print(f'\n🏗️ 3D OBJ file: {obj_path} (open in Blender/MeshLab)')